In [1]:
import os
import sys
import pandas as pd
import yfinance as yf
import plotly as px

In [2]:
import os, sys
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from statsmodels.tsa.seasonal import seasonal_decompose
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

In [3]:


# Get theconda create --name talib python=3.9 current working directory
current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# # Insert the path to the Scripts directory
# sys.path.insert(0, os.path.join(parent_dir, 'Scripts'))

# print(sys.path)

d:\Kifya\Week01\notebooks
d:\Kifya\Week01


In [4]:
# nltk.download('vader_lexicon')

In [5]:
news_data = pd.read_csv('../yfinance_data//raw_analyst_ratings.csv')
# Remove timezone information and convert to datetime

# Step 1: Convert the 'date' column to datetime format
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')

# Step 2: Remove timezone information (if any)
news_data['local_date'] = news_data['date'].dt.tz_localize(None)
news_data['date_only'] = news_data['date'].dt.date
news_data

,Unnamed: 0,headline,url,publisher,date,stock,local_date,date_only
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,2020-06-05
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 10:45:20,2020-06-03
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 04:30:07,2020-05-26
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 12:45:06,2020-05-22
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 11:38:59,2020-05-22
...,...,...,...,...,...,...,...,...
1407323,1413844,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,NaT,ZX,NaT,NaT
1407324,1413845,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,NaT,ZX,NaT,NaT
1407325,1413846,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,NaT,ZX,NaT,NaT
1407326,1413847,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,NaT,ZX,NaT,NaT


In [6]:
# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()
# Calculate the sentiment of the headlines
news_data['sentiment'] = news_data['headline'].apply(lambda x: sia.polarity_scores(text=x)['compound'])


# Initialize VADER sentiment analyzer
# Categorize the sentiment scores
news_data['sentiment_category'] = pd.cut(news_data['sentiment'], bins=[-1, -0.5, -0.0001, 0.5, 1], labels=['Very Negative', 'Negative', 'Neutral', 'Positive'])
news_data.head()

,Unnamed: 0,headline,url,publisher,date,stock,local_date,date_only,sentiment,sentiment_category
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,2020-06-05,0.000,Neutral
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 10:45:20,2020-06-03,0.000,Neutral
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 04:30:07,2020-05-26,0.000,Neutral
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 12:45:06,2020-05-22,0.000,Neutral
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 11:38:59,2020-05-22,0.296,Neutral


In [13]:
import pandas as pd

def count_stocks(news_data, stocks):

    stock_counts = {}
    for stock in stocks:
        stock_counts[stock] = len(news_data[news_data['stock'] == stock])
    return stock_counts

# Example usage:
stocks_to_count = ['AAPL', 'GOOG', 'MSFT', 'META','AMZN','NVDA','TSLA']
stock_counts = count_stocks(news_data, stocks_to_count)
print(stock_counts)

{'AAPL': 441, 'GOOG': 1199, 'MSFT': 0, 'META': 0, 'AMZN': 278, 'NVDA': 3146, 'TSLA': 1875}


exclude 'MSFT': 0, 'META': 0,for correlation

In [15]:
# news_data=news_data.drop(columns='Unnamed: 0', axis=1)
# news_data.head(1)

In [16]:
news_data.reset_index(inplace=True)

In [17]:
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')

# Step 2: Remove timezone information (if any)
# news_data['local_date'] = news_data['date'].dt.tz_localize(None)
news_data['Date'] = news_data['date'].dt.date

In [18]:
news_data.head(1)

,Date,index,headline,url,publisher,date,stock,local_date,date_only,sentiment,sentiment_category
0,2020-06-05,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,2020-06-05,0.0,Neutral


In [19]:
# news_data['Date'] = pd.to_datetime(news_data['date_only'])
news_data.set_index('Date', inplace=True)
news_data.head(1)

,index,headline,url,publisher,date,stock,local_date,date_only,sentiment,sentiment_category
Date,,,,,,,,,,
2020-06-05,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,2020-06-05,0.0,Neutral


### Stocks data

In [33]:

# Load data
tesla_df = pd.read_csv('../yfinance_data/TSLA_historical_data.csv')
goog_df = pd.read_csv('../yfinance_data/GOOG_historical_data.csv')
amzn_df = pd.read_csv('../yfinance_data/AMZN_historical_data.csv')
nvda_df = pd.read_csv('../yfinance_data/NVDA_historical_data.csv')
aapl_df = pd.read_csv('../yfinance_data/AAPL_historical_data.csv')

In [34]:
# Convert Date columns to datetime
amzn_df['Date'] = pd.to_datetime(amzn_df['Date']).dt.date
tesla_df['Date'] = pd.to_datetime(tesla_df['Date']).dt.date
goog_df['Date'] = pd.to_datetime(goog_df['Date']).dt.date
nvda_df['Date'] = pd.to_datetime(nvda_df['Date']).dt.date
aapl_df['Date'] = pd.to_datetime(aapl_df['Date']).dt.date

In [38]:
# Calculate daily stock returns for each DataFrame
amzn_df['AMZN_Daily_Return'] = amzn_df['Close'].pct_change()
tesla_df['TSLA_Daily_Return'] = tesla_df['Close'].pct_change()
goog_df['GOOG_Daily_Return'] = goog_df['Close'].pct_change()
nvda_df['NVDA_Daily_Return'] = nvda_df['Close'].pct_change()
aapl_df['AAPL_Daily_Return'] = aapl_df['Close'].pct_change()

### First Pearson correlation coefficient  b/n Tesla and news

In [54]:
tesla_df.head(1)

,index,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,TSLA_Daily_Return
0,1970-01-01,2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500,0.0,0.0,NaN


In [48]:
# Aggregate sentiment scores by date
daily_sentiment = news_data.groupby('date')['sentiment'].mean().reset_index()
daily_sentiment.head(1)

,date,sentiment
0,2011-04-27 21:01:48-04:00,0.0


In [51]:
# Convert the index to datetime (since Date is already the index)
tesla_df.index = pd.to_datetime(tesla_df.index).date
news_data['date'] = pd.to_datetime(news_data['date']).dt.date

# Calculate daily returns for Tesla stock
tesla_df['TSLA_Daily_Return'] = tesla_df['Adj Close'].pct_change()

# Aggregate sentiment scores by date
daily_sentiment = news_data.groupby('date')['sentiment'].mean().reset_index()

# Reset index to make 'Date' a column for merging
tesla_df.reset_index(inplace=True)

# Merge the daily sentiment scores with Tesla's daily returns
merged_data = pd.merge(daily_sentiment, tesla_df[['Date', 'TSLA_Daily_Return']], left_on='date', right_on='Date')

# Calculate the Pearson correlation coefficient
correlation = merged_data['sentiment'].corr(merged_data['TSLA_Daily_Return'])

print(f'Correlation between sentiment and Tesla stock returns: {correlation}')

Correlation between sentiment and Tesla stock returns: -0.01581691000688443


## For ALL STOCKS Pearson correlation coefficient 

In [61]:
# Aggregate sentiment scores by date
# daily_sentiment = news_data.groupby('date')['sentiment'].mean().reset_index()

# Reset index to make 'Date' a column for merging
amzn_df.reset_index(inplace=True)
tesla_df.reset_index(inplace=True)
goog_df.reset_index(inplace=True)
nvda_df.reset_index(inplace=True)
aapl_df.reset_index(inplace=True)

In [64]:
# Merge the daily sentiment scores with each stock's daily returns
merged_amzn = pd.merge(daily_sentiment, amzn_df[['Date', 'AMZN_Daily_Return']], left_on='date', right_on='Date')
merged_tesla = pd.merge(daily_sentiment, tesla_df[['Date', 'TSLA_Daily_Return']], left_on='date', right_on='Date')
merged_goog = pd.merge(daily_sentiment, goog_df[['Date', 'GOOG_Daily_Return']], left_on='date', right_on='Date')
merged_nvda = pd.merge(daily_sentiment, nvda_df[['Date', 'NVDA_Daily_Return']], left_on='date', right_on='Date')
merged_aapl = pd.merge(daily_sentiment, aapl_df[['Date', 'AAPL_Daily_Return']], left_on='date', right_on='Date')


In [65]:
# Calculate the Pearson correlation coefficient
correlation_amzn = merged_amzn['sentiment'].corr(merged_amzn['AMZN_Daily_Return'])
correlation_tesla = merged_tesla['sentiment'].corr(merged_tesla['TSLA_Daily_Return'])
correlation_goog = merged_goog['sentiment'].corr(merged_goog['GOOG_Daily_Return'])
correlation_nvda = merged_nvda['sentiment'].corr(merged_nvda['NVDA_Daily_Return'])
correlation_aapl = merged_aapl['sentiment'].corr(merged_aapl['AAPL_Daily_Return'])

In [66]:
print(f'Correlation between sentiment and Amazon stock returns: {correlation_amzn}')
print(f'Correlation between sentiment and Tesla stock returns: {correlation_tesla}')
print(f'Correlation between sentiment and Google stock returns: {correlation_goog}')
print(f'Correlation between sentiment and NVIDIA stock returns: {correlation_nvda}')
print(f'Correlation between sentiment and Apple stock returns: {correlation_aapl}')

Correlation between sentiment and Amazon stock returns: -0.0015263845654808995
Correlation between sentiment and Tesla stock returns: -0.01581691000688443
Correlation between sentiment and Google stock returns: 0.02468864641328575
Correlation between sentiment and NVIDIA stock returns: 0.02673313534292091
Correlation between sentiment and Apple stock returns: 0.04535919201222875


In [67]:
def calculate_correlation(daily_sentiment, stock_df, stock_name):
    merged_data = pd.merge(daily_sentiment, stock_df[['Date', f'{stock_name}_Daily_Return']], 
                           left_on='date', right_on='Date')
    correlation = merged_data['sentiment'].corr(merged_data[f'{stock_name}_Daily_Return'])
    print(f'Correlation between sentiment and {stock_name} stock returns: {correlation}')

# Calculate correlations for all stocks
calculate_correlation(daily_sentiment, amzn_df, 'AMZN')
calculate_correlation(daily_sentiment, tesla_df, 'TSLA')
calculate_correlation(daily_sentiment, goog_df, 'GOOG')
calculate_correlation(daily_sentiment, nvda_df, 'NVDA')
calculate_correlation(daily_sentiment, aapl_df, 'AAPL')

Correlation between sentiment and AMZN stock returns: -0.0015263845654808995
Correlation between sentiment and TSLA stock returns: -0.01581691000688443
Correlation between sentiment and GOOG stock returns: 0.02468864641328575
Correlation between sentiment and NVDA stock returns: 0.02673313534292091
Correlation between sentiment and AAPL stock returns: 0.04535919201222875


In [68]:
# Display the first few rows of each DataFrame to confirm the calculations
print(amzn_df[['Date', 'Close', 'AMZN_Daily_Return']].head())
print(tesla_df[['Date', 'Close', 'TSLA_Daily_Return']].head())
print(goog_df[['Date', 'Close', 'GOOG_Daily_Return']].head())
print(nvda_df[['Date', 'Close', 'NVDA_Daily_Return']].head())
print(aapl_df[['Date', 'Close', 'AAPL_Daily_Return']].head())

         Date     Close  AMZN_Daily_Return
0  1997-05-15  0.097917                NaN
1  1997-05-16  0.086458          -0.117028
2  1997-05-19  0.085417          -0.012040
3  1997-05-20  0.081771          -0.042685
4  1997-05-21  0.071354          -0.127392
         Date     Close  TSLA_Daily_Return
0  2010-06-29  1.592667                NaN
1  2010-06-30  1.588667          -0.002511
2  2010-07-01  1.464000          -0.078473
3  2010-07-02  1.280000          -0.125683
4  2010-07-06  1.074000          -0.160937
         Date     Close  GOOG_Daily_Return
0  2004-08-19  2.499133                NaN
1  2004-08-20  2.697639           0.079430
2  2004-08-23  2.724787           0.010064
3  2004-08-24  2.611960          -0.041408
4  2004-08-25  2.640104           0.010775
         Date     Close  NVDA_Daily_Return
0  1999-01-22  0.041016                NaN
1  1999-01-25  0.045313           0.104764
2  1999-01-26  0.041797          -0.077594
3  1999-01-27  0.041667          -0.003110
4  1999-01-